In [ ]:
import pandas as pd
import copy
import math
import numpy as np
import enum 

import networkx as nx
from networkx.convert_matrix import from_numpy_array

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

import scipy.spatial.distance
import scipy.stats as stats
from scipy.stats import pearsonr
from scipy.stats import kurtosis, skew

import matplotlib.pyplot as plt

import pylab

: 

In [ ]:
cell_data = pd.read_csv('cell_data.tsv', sep='\t')
cell_data.rename(columns={cell_data.columns[0]: 'cellID'}, inplace=True)
cell_data

: 

**Formiranje matrica susedstva za grafove G1 i G2 kod kojih su cvorovi celije, a tezine grana:**
- euklidsko rastojanje na osnovu koordinata celija (x,y)
- euklidsko rastojanje na osnovu redukovanih genskih ekspresija celija

In [ ]:
# pravi matricu susedstva grafa 
# data_dict[celija] = (x,y) ili data_dict[celija] = redukovane genske ekspresije
# cells = [id_celije_1, ... , id_celije_n]
def adjacency_matrix(data_dict, cells):
    n = len(cells)
    adjacency_matrix = np.zeros(shape=(n, n))

    for i in range(0, n - 1):
        for j in range(i + 1, n):
            adjacency_matrix[i][j] = scipy.spatial.distance.euclidean(data_dict[cells[i]], data_dict[cells[j]])
            adjacency_matrix[j][i] = adjacency_matrix[i][j]

    return adjacency_matrix

: 

In [ ]:
cell_dict = {}
cells = []

# cell_dict ce sadrzati koordinate celija (cvorova) 
# cells je lista id-eva svih celija
for index, row in cell_data.iterrows():
    cell_dict[row['cellID']] = (row['x'], row['y'])
    cells.append(row['cellID'])

: 

In [ ]:
adjacency_matrix_1 = adjacency_matrix(cell_dict, cells)

: 

In [ ]:
# Redukcija dimenzionalnosti genskih ekspresija pomocu PCA metode
gene_exp = cell_data.copy()
gene_exp.drop('cellID', axis=1, inplace=True)
gene_exp.drop('x', axis=1, inplace=True)
gene_exp.drop('y', axis=1, inplace=True)

scaling = StandardScaler()
scaling.fit(gene_exp)
scaled_data = scaling.transform(gene_exp)

pca = PCA(n_components=2500)
reduced_data = pca.fit_transform(scaled_data)
pca.explained_variance_ratio_.cumsum()

: 

In [ ]:
# reduced_data_dict ce sadrzati redukovane genske ekspresije celija (cvorova) 
reduced_data_dict = {}
for i in range(0, len(reduced_data)):
    reduced_data_dict[cells[i]] = reduced_data[i]
len(reduced_data_dict)

: 

In [ ]:
adjacency_matrix_2 = adjacency_matrix(reduced_data_dict, cells)

: 

**Normalizovanje matrica susedstva grafova G1 i G2.**

In [ ]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
norm_adj_matrix_1 = min_max_scaler.fit_transform(adjacency_matrix_1)
norm_adj_matrix_1

: 

In [ ]:
norm_adj_matrix_2 = min_max_scaler.fit_transform(adjacency_matrix_2)
norm_adj_matrix_2

: 

**Poredjenje celija na osnovu svih atributa celije (x, y, redukovane genske ekspresije)**

Formiranje recnika koji ima za kljuceve id celuje, a za vrednosti vektor (x,y, redukovane genske ekspresije)

In [ ]:
cells_all_features = {}
for cell in cells:
    list1 = np.array(cell_dict[cell])
    list2 = np.array(reduced_data_dict[cell])
    cells_all_features[cell] = np.concatenate((list1, list2))
cells_all_features

: 

In [ ]:
import igraph as ig
import leidenalg
import matplotlib.cm as cm

: 

In [ ]:
def reduce_graph(adjacency_matrix, N):
   G = ig.Graph.Weighted_Adjacency(adjacency_matrix.tolist(), mode=ig.ADJ_UPPER)

   reduced_G = ig.Graph()
   reduced_G.add_vertices(G.vcount())

   for node in G.vs:
     neighbors = sorted(G.neighbors(node, mode="out"), key=lambda n: G.es[G.get_eid(node.index, n)]['weight'])[:N]
     for neighbor in neighbors:
        reduced_G.add_edge(node.index, neighbor, weight=G.es[G.get_eid(node.index, neighbor)]['weight'])
   
   return reduced_G

: 

In [ ]:
def plot(G, node_colors, cell_dict, cells):
    fig, ax = plt.subplots()

    for node in G.vs:
        (x, y) = cell_dict[cells[node.index]]
        color = node_colors[node.index]
        ax.scatter(x, y, color=color, s=50, alpha=0.7)

    ax.axis("off")
    plt.show()

: 

In [ ]:
def plot_sec(G, node_colors):
    layout = G.layout(layout_algorithm="fruchterman_reingold")
    coords = np.array(layout)

    plt.scatter(coords[:, 0], coords[:, 1], s=50, color=node_colors, alpha=0.7)

    plt.axis("off")
    plt.show()

: 

In [ ]:
def make_union(reduced_graph_1, reduced_graph_2):
    G_union = ig.Graph(len(reduced_graph_1.vs), directed=False)

    # Dodavanje grana iz prvog grafa
    for edge in reduced_graph_1.es:
        source, target = edge.tuple
        G_union.add_edge(source, target, weight=edge["weight"])

    # Dodavanje grana iz drugog grafa, kombinovanje težina
    for edge in reduced_graph_2.es:
        source, target = edge.tuple
        if G_union.are_connected(source, target):
            existing_edge = G_union.es[G_union.get_eid(source, target)]
            existing_edge["weight"] = (existing_edge["weight"] + edge["weight"]) / 2
        else:
            G_union.add_edge(source, target, weight=edge["weight"])
    
    return G_union

: 

In [ ]:
def cluster_lo(reduced1, reduced2, cell_dict, cells):
    G_union = make_union(reduced1, reduced2)

    clusters = G_union.community_multilevel(weights=G_union.es["weight"])
    
    modularity = G_union.modularity(clusters)

    num_clusters = len(set(clusters.membership))
    print("Optimalna modularnost:", modularity)
    print("Broj klastera:", num_clusters)
    print("\n")

    color_map = cm.get_cmap('tab20c', num_clusters)
    colors = [color_map(i) for i in range(num_clusters)]

    node_colors = [colors[cluster] for cluster in  clusters.membership]

    plot(G_union, node_colors, cell_dict, cells)

: 

In [ ]:
def cluster_leiden_modularity(reduced1, reduced2):
    G_union = make_union(reduced1, reduced2)

    partition = leidenalg.find_partition(graph=G_union, partition_type=leidenalg.ModularityVertexPartition, n_iterations=-1)

    print("Optimalna modularnost:", partition.modularity)
    print("Broj klastera:", len(set(partition.membership)))
    print("\n")

    color_map = cm.get_cmap('tab20c', len(set(partition.membership)))
    colors = [color_map(i) for i in range(len(set(partition.membership)))]

    node_colors = [colors[cluster] for cluster in  partition.membership]

    plot(G_union, node_colors, cell_dict, cells)
    plot_sec(G_union, node_colors)

    return partition


: 

In [ ]:
def cluster_leiden_cpm(reduced1, reduced2):
    G_union = make_union(reduced1, reduced2)

    partition = leidenalg.find_partition(graph=G_union, partition_type=leidenalg.CPMVertexPartition, n_iterations=-1)

    print("Optimalna modularnost (2):", partition.modularity)
    print("Broj klastera (2):", len(set(partition.membership)))
    print("\n")

    color_map = cm.get_cmap('tab20c', len(set(partition.membership)))
    colors = [color_map(i) for i in range(len(set(partition.membership)))]

    node_colors = [colors[cluster] for cluster in  partition.membership]

    plot(G_union, node_colors, cell_dict, cells)
    plot_sec(G_union, node_colors)

    return partition
    

: 

In [ ]:
def optimize_partition(partition, reduced1, reduced2):
    G_union = make_union(reduced1, reduced2)

    optimiser = leidenalg.Optimiser()
    steps = 1
    diff = 1
    
    while diff > 0:
        diff = optimiser.optimise_partition(partition)
        steps += 1

    print("Number of steps: " + str(steps))

    print("Optimalna modularnost:", partition.modularity)
    print("Broj klastera:", len(set(partition.membership)))
    print("\n")

    color_map = cm.get_cmap('tab20c', len(set(partition.membership)))
    colors = [color_map(i) for i in range(len(set(partition.membership)))]

    node_colors = [colors[cluster] for cluster in  partition.membership]

    plot(G_union, node_colors, cell_dict, cells)
    plot_sec(G_union, node_colors) 

: 

**LEIDEN**

In [ ]:
i=15
print("\n--------------------------------------------\n")
print("Broj najblizih suseda: "+ str(i))

reduced_graph_1 = reduce_graph(norm_adj_matrix_1, i)
reduced_graph_2 = reduce_graph(norm_adj_matrix_2, i)

pm = cluster_leiden_modularity(reduced_graph_1, reduced_graph_2)
pc = cluster_leiden_cpm(reduced_graph_1, reduced_graph_2)

optimize_partition(pm, reduced_graph_1, reduced_graph_2)
optimize_partition(pc, reduced_graph_1, reduced_graph_2)

: 